In [1]:
!pip install --upgrade telethon


In [2]:
from telethon import TelegramClient, events, sync, functions
import numpy as np
import pandas as pd
import os
from dotenv import load_dotenv

Вот здесь прямо под 3 ячейкой появится строка ввода идентификатора сессии, который придет в личку в телеграме юзеру, апи и хэш которого ты будешь использовать (по крайней мере при создании первой сессии).  Как получить айди и хэщ https://blog.lavhost.ml/telegram-api

In [3]:
dotenv_path = os.path.join('dot.env')
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)

In [4]:
# Set up a new Telegram client session
api_id = os.environ['api_id']
api_hash = os.environ['api_hash']
client = TelegramClient('my_session', api_id, api_hash)


Через бота все бы проще получалось, НО - БОТ должен быть официально зарегистрирован в качестве подписчика канала, а без разрешения админа - вот вообще никак. Поэтому пилила через client

In [10]:
data = []
all_reactions = []
media = []
forwarded = []
replies = []
await client.start() 


channels_name = ['https://t.me/kinopoisk']

for name in channels_name:
    channel_entity = await client.get_entity(name)
    async for message in client.iter_messages(channel_entity, limit = None):
        reactions = message.reactions 
        data.append([name, message.id, message.date, message.text, message.views])
        
        try:
            all_reactions.append(reactions.to_dict())
        except:
            all_reactions.append(np.nan)
            
        try:
            media.append(message.media.to_dict())
        except:
            media.append(np.nan)
            
            
        try:
            forwarded.append(message.forwards)
        except:
            forwarded.append(np.nan)
            
        try:
            replies.append(message.replies.to_dict())
        except:
            replies.append(np.nan)
            
       
            
            
        

In [11]:
df = pd.DataFrame(data, columns = ['channel','id', 'date', 'text', 'views'])
df['reactions'] = all_reactions
df['with_media'] = media
df['forwarded'] = forwarded
df['replies'] = replies

In [12]:
df

,channel,id,date,text,views,reactions,with_media,forwarded,replies
0,https://t.me/kinopoisk,37125,2024-01-21 08:02:21+00:00,Какими подростками были актеры из «Трудных под...,12744.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaWebPage', 'webpage': {'_': ...",12.0,"{'_': 'MessageReplies', 'replies': 5, 'replies..."
1,https://t.me/kinopoisk,37124,2024-01-20 18:01:00+00:00,,50014.0,NaN,"{'_': 'MessageMediaPhoto', 'spoiler': False, '...",161.0,NaN
2,https://t.me/kinopoisk,37123,2024-01-20 18:01:00+00:00,,49690.0,NaN,"{'_': 'MessageMediaPhoto', 'spoiler': False, '...",161.0,NaN
3,https://t.me/kinopoisk,37122,2024-01-20 18:01:00+00:00,,49426.0,NaN,"{'_': 'MessageMediaPhoto', 'spoiler': False, '...",161.0,NaN
4,https://t.me/kinopoisk,37121,2024-01-20 18:01:00+00:00,Фильм дня — [**«Дневник Бриджит Джонс»**](http...,49486.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaPhoto', 'spoiler': False, '...",161.0,"{'_': 'MessageReplies', 'replies': 85, 'replie..."
...,...,...,...,...,...,...,...,...,...
35653,https://t.me/kinopoisk,11,2016-12-21 08:40:33+00:00,"И, кстати, у нас есть первая тысяча подписчико...",5967.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...",NaN,0.0,NaN
35654,https://t.me/kinopoisk,10,2016-12-21 08:35:05+00:00,Первые счастливчики уже посмотрели новый эпизо...,6486.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...",NaN,4.0,NaN
35655,https://t.me/kinopoisk,9,2016-12-20 08:18:29+00:00,"Каждый год кто-то ходит в баню, а мы выбираем ...",8188.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaWebPage', 'webpage': {'_': ...",11.0,NaN
35656,https://t.me/kinopoisk,8,2016-12-19 16:24:41+00:00,Всем привет! Меня зовут Лиза Сурганова и я воз...,9078.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...",NaN,16.0,NaN


In [13]:
df = df.query('text != "" and text.notna()')

In [ ]:
df.info()

Небольшое описание таблицы

- channel - телеграм канал
- id - айдишник поста в этом канале
- date - дата публикации
- text - текстовое содержание поста
- views - количество просмотров
- reactions - эможди - типа json формата, отображает какой смайл сколько раз был поставлен, принимает значение NaN если реакций не было
- with_media - показывает было ли прикреплено к посту фото/видео, принимает значение Nan, если в посте только текст
- forwarded - был ли пост переслан (не знаю, как по-русски), если нет - то Nan
- replies - комментарии, причем подсчитываются только комменты, оставленные под постом подписчиками, но не ответы (реплаи) на эти комменты

In [20]:
def get_number_reactions(cell):
    try:
        count = 0
        for i in range (len(cell['results'])):
            count += cell['results'][i]['count']
        return count
    except: return 0
df['reactions_count'] = df['reactions'].apply(get_number_reactions)

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_12004\3840059049.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reactions_count'] = df['reactions'].apply(get_number_reactions)


In [21]:
from ast import literal_eval
def get_comments(cell):
    try:
        return cell['replies']
    except:
        return 0
df['comments'] = df['replies'].apply(get_comments)

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_12004\3890207502.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comments'] = df['replies'].apply(get_comments)


In [22]:
def get_type_attachment(cell):
    try:
        return cell['_']
    except:
        return np.nan
df['type_attachment'] = df['with_media'].apply(get_type_attachment)

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_12004\1935664372.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['type_attachment'] = df['with_media'].apply(get_type_attachment)


In [23]:
res = df.reset_index()
res

,index,channel,id,date,text,views,reactions,with_media,forwarded,replies,reactions_count,comments,type_attachment
0,0,https://t.me/kinopoisk,37125,2024-01-21 08:02:21+00:00,Какими подростками были актеры из «Трудных под...,12744.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaWebPage', 'webpage': {'_': ...",12.0,"{'_': 'MessageReplies', 'replies': 5, 'replies...",57,5,MessageMediaWebPage
1,4,https://t.me/kinopoisk,37121,2024-01-20 18:01:00+00:00,Фильм дня — [**«Дневник Бриджит Джонс»**](http...,49486.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaPhoto', 'spoiler': False, '...",161.0,"{'_': 'MessageReplies', 'replies': 85, 'replie...",896,85,MessageMediaPhoto
2,5,https://t.me/kinopoisk,37120,2024-01-20 16:01:16+00:00,Кристоферу Нолану [вручат](https://www.hollywo...,53713.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaPhoto', 'spoiler': False, '...",136.0,"{'_': 'MessageReplies', 'replies': 68, 'replie...",1782,68,MessageMediaPhoto
3,6,https://t.me/kinopoisk,37119,2024-01-20 14:20:04+00:00,Отгадайте фильм: в жизни грустной девочки появ...,55890.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaWebPage', 'webpage': {'_': ...",148.0,"{'_': 'MessageReplies', 'replies': 12, 'replie...",189,12,MessageMediaWebPage
4,7,https://t.me/kinopoisk,37118,2024-01-20 10:48:48+00:00,Правда или фейк? 🧐\n\n#ДежурныйПоКинопоиску Се...,60348.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaDocument', 'nopremium': Fal...",27.0,"{'_': 'MessageReplies', 'replies': 3, 'replies...",210,3,MessageMediaDocument
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23321,35651,https://t.me/kinopoisk,13,2016-12-21 10:02:22+00:00,"Кстати, уже через три часа, в 16-00 по Москве...",5990.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...",NaN,0.0,NaN,3,0,NaN
23322,35653,https://t.me/kinopoisk,11,2016-12-21 08:40:33+00:00,"И, кстати, у нас есть первая тысяча подписчико...",5967.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...",NaN,0.0,NaN,7,0,NaN
23323,35654,https://t.me/kinopoisk,10,2016-12-21 08:35:05+00:00,Первые счастливчики уже посмотрели новый эпизо...,6486.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...",NaN,4.0,NaN,2,0,NaN
23324,35655,https://t.me/kinopoisk,9,2016-12-20 08:18:29+00:00,"Каждый год кто-то ходит в баню, а мы выбираем ...",8188.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaWebPage', 'webpage': {'_': ...",11.0,NaN,6,0,MessageMediaWebPage


In [27]:
df.query('id == 36939')

,channel,id,date,text,views,reactions,with_media,forwarded,replies,reactions_count,comments,type_attachment
182,https://t.me/kinopoisk,36939,2024-01-08 17:06:10+00:00,Устроили киномарафон в выходные или решили пос...,94544.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaPhoto', 'spoiler': False, '...",28.0,"{'_': 'MessageReplies', 'replies': 262, 'repli...",297,262,MessageMediaPhoto


In [28]:
df['date'].max()

Timestamp('2024-01-21 08:02:21+0000', tz='UTC')

In [29]:
df.head()

,channel,id,date,text,views,reactions,with_media,forwarded,replies,reactions_count,comments,type_attachment
0,https://t.me/kinopoisk,37125,2024-01-21 08:02:21+00:00,Какими подростками были актеры из «Трудных под...,12744.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaWebPage', 'webpage': {'_': ...",12.0,"{'_': 'MessageReplies', 'replies': 5, 'replies...",57,5,MessageMediaWebPage
4,https://t.me/kinopoisk,37121,2024-01-20 18:01:00+00:00,Фильм дня — [**«Дневник Бриджит Джонс»**](http...,49486.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaPhoto', 'spoiler': False, '...",161.0,"{'_': 'MessageReplies', 'replies': 85, 'replie...",896,85,MessageMediaPhoto
5,https://t.me/kinopoisk,37120,2024-01-20 16:01:16+00:00,Кристоферу Нолану [вручат](https://www.hollywo...,53713.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaPhoto', 'spoiler': False, '...",136.0,"{'_': 'MessageReplies', 'replies': 68, 'replie...",1782,68,MessageMediaPhoto
6,https://t.me/kinopoisk,37119,2024-01-20 14:20:04+00:00,Отгадайте фильм: в жизни грустной девочки появ...,55890.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaWebPage', 'webpage': {'_': ...",148.0,"{'_': 'MessageReplies', 'replies': 12, 'replie...",189,12,MessageMediaWebPage
7,https://t.me/kinopoisk,37118,2024-01-20 10:48:48+00:00,Правда или фейк? 🧐\n\n#ДежурныйПоКинопоиску Се...,60348.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaDocument', 'nopremium': Fal...",27.0,"{'_': 'MessageReplies', 'replies': 3, 'replies...",210,3,MessageMediaDocument


In [30]:
df.to_csv('kinopoisk_channel_posts_2023-01-21.csv')

In [31]:
pd.read_csv('kinopoisk_channel_posts_2023-01-21.csv')

,Unnamed: 0,channel,id,date,text,views,reactions,with_media,forwarded,replies,reactions_count,comments,type_attachment
0,0,https://t.me/kinopoisk,37125,2024-01-21 08:02:21+00:00,Какими подростками были актеры из «Трудных под...,12744.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaWebPage', 'webpage': {'_': ...",12.0,"{'_': 'MessageReplies', 'replies': 5, 'replies...",57,5,MessageMediaWebPage
1,4,https://t.me/kinopoisk,37121,2024-01-20 18:01:00+00:00,Фильм дня — [**«Дневник Бриджит Джонс»**](http...,49486.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaPhoto', 'spoiler': False, '...",161.0,"{'_': 'MessageReplies', 'replies': 85, 'replie...",896,85,MessageMediaPhoto
2,5,https://t.me/kinopoisk,37120,2024-01-20 16:01:16+00:00,Кристоферу Нолану [вручат](https://www.hollywo...,53713.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaPhoto', 'spoiler': False, '...",136.0,"{'_': 'MessageReplies', 'replies': 68, 'replie...",1782,68,MessageMediaPhoto
3,6,https://t.me/kinopoisk,37119,2024-01-20 14:20:04+00:00,Отгадайте фильм: в жизни грустной девочки появ...,55890.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaWebPage', 'webpage': {'_': ...",148.0,"{'_': 'MessageReplies', 'replies': 12, 'replie...",189,12,MessageMediaWebPage
4,7,https://t.me/kinopoisk,37118,2024-01-20 10:48:48+00:00,Правда или фейк? 🧐\n\n#ДежурныйПоКинопоиску Се...,60348.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaDocument', 'nopremium': Fal...",27.0,"{'_': 'MessageReplies', 'replies': 3, 'replies...",210,3,MessageMediaDocument
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23321,35651,https://t.me/kinopoisk,13,2016-12-21 10:02:22+00:00,"Кстати, уже через три часа, в 16-00 по Москве...",5990.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...",NaN,0.0,NaN,3,0,NaN
23322,35653,https://t.me/kinopoisk,11,2016-12-21 08:40:33+00:00,"И, кстати, у нас есть первая тысяча подписчико...",5967.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...",NaN,0.0,NaN,7,0,NaN
23323,35654,https://t.me/kinopoisk,10,2016-12-21 08:35:05+00:00,Первые счастливчики уже посмотрели новый эпизо...,6486.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...",NaN,4.0,NaN,2,0,NaN
23324,35655,https://t.me/kinopoisk,9,2016-12-20 08:18:29+00:00,"Каждый год кто-то ходит в баню, а мы выбираем ...",8188.0,"{'_': 'MessageReactions', 'results': [{'_': 'R...","{'_': 'MessageMediaWebPage', 'webpage': {'_': ...",11.0,NaN,6,0,MessageMediaWebPage
